In [1]:
# Turn off warnings 
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')



In [305]:
## Cleaning supplies! 

from nltk.corpus import stopwords
import string
from nltk.util import ngrams
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

import spacy

from textblob import TextBlob

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [3]:
## loading data
df = pd.read_csv('data/reviews.csv', index_col='Unnamed: 0')

# Remove nan text
df.dropna(subset=['Review Text'], inplace= True)

In [4]:
df.shape

(22641, 10)

In [105]:
df.corr()

,Clothing ID,Age,Rating,Recommended IND,Positive Feedback Count
Clothing ID,1.000000,0.017688,-0.018454,-0.014874,0.044902
Age,0.017688,1.000000,0.029962,0.034208,0.040850
Rating,-0.018454,0.029962,1.000000,0.792568,-0.060984
Recommended IND,-0.014874,0.034208,0.792568,1.000000,-0.065923
Positive Feedback Count,0.044902,0.040850,-0.060984,-0.065923,1.000000


In [104]:
df['Review Text'].iloc[150]

'This top is so much better in person. i do not agree with some of the other reviews about the fabric being scratchy. it is not and i have sensitive skin. i love this top and have got lots of compliments.'

In [243]:
# Corpus
corpus = df[['Review Text']].values.tolist()
corpus = [''.join(element) for element in corpus]

In [244]:
corpus[1]

'Love this dress!  it\'s sooo pretty.  i happened to find it in a store, and i\'m glad i did bc i never would have ordered it online bc it\'s petite.  i bought a petite and am 5\'8".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.'

In [245]:
#corpus is a list of strings
type(corpus)

list

In [204]:
my_stopwords = ['i','me','my','myself','we','our','ours','ourselves','you',"you're","you've","you'll","you'd",'your',\
                'yours','yourself','yourselves','he','him','his','himself','she',"she's",'her','hers','herself',\
                'it',"it's",'its','itself','they','them','their','theirs','themselves','what','which','who','whom',\
                'this','that',"that'll",'these','those','am','is','are','was','were','be','been','being','have',\
                'has','had','having','do','does','did','doing' 'a','an','the','and','but','if','or','because','as',\
                'until','while','of','at','by','for','with','about','against','between','into','during','before',\
                'after','to','from','in','out','on','off','over','under','again','further','then','once',\
                'here','there','when','where','why','how','all','any','both','each','few','more','most','other',\
                'some','such','nor','only','own','same','so','than','too','very','s','t','can','will','just','don',\
                "don't",'should',"should've",'now','d','ll','m','o','re','ve','y','ain','aren',"aren't",'couldn',\
                "couldn't",'could','a',"'s", "'m",\
                'didn',"didn't",'doesn',"doesn't",'hadn',"hadn't",'hasn',"hasn't",'haven', "haven't",'isn',"isn't",\
                'ma','mightn',"mightn't",'mustn',"mustn't",'needn',"needn't",'shan',"shan't",'shouldn',"shouldn't",\
                'wasn',"wasn't",'weren',"weren't",'won',"won't",'wouldn',"wouldn't", 'would']



In [109]:
pt = string.punctuation + "''"
pt

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\'\''

In [34]:
contractions = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"isn't": "is not",
"mayn't": "may not",
"mightn't": "might not",
"mightn't've": "might not have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"wasn't": "was not",
"weren't": "were not",
"won't": "will not",
"won't've": "will not have",
"wouldn't": "would not",
"wouldn't've": "would not have",
}

In [35]:
text="What's the best way to ensure this?"
for word in text.split():
    if word.lower() in contractions:
        text = text.replace(word, contractions[word.lower()])
print(text)

What's the best way to ensure this?


In [248]:
def clean_nots(docs):
    not_docs=[]
    for doc in docs:
        for word in doc.split():
            if word in contractions:
                doc = doc.replace(word,contractions[word])
        not_docs.append(doc)
    return not_docs


In [249]:
corpus[7]

"I ordered this in carbon for store pick up, and had a ton of stuff (as always) to try on and used this top to pair (skirts and pants). everything went with it. the color is really nice charcoal with shimmer, and went well with pencil skirts, flare pants, etc. my only compaint is it is a bit big, sleeves are long and it doesn't go in petite. also a bit loose for me, but no xxs... so i kept it and wil ldecide later since the light color is already sold out in hte smallest size..."

In [252]:
corpus_w_nots = clean_nots(corpus)

In [256]:
corpus_w_nots[0:2]

['Absolutely wonderful - silky and sexy and comfortable',
 'Love this dress!  it\'s sooo pretty.  i happened to find it in a store, and i\'m glad i did bc i never would have ordered it online bc it\'s petite.  i bought a petite and am 5\'8".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.']

In [177]:
def clean_tokens(docs):
    ''' returns a list of lists: the clean tokens of docs'''
    docs_tokens=[]
    for doc in docs:
        clean_tokens = []
        tokens = word_tokenize(doc)
        for token in tokens:
            if token.lower() not in my_stopwords and token not in pt:
                clean_tokens.append(token.lower())
        docs_tokens.append(clean_tokens)    
    return docs_tokens

In [257]:
c_tokens = clean_docs(corpus_w_nots)


In [355]:
#c_tokens is a list of token lists for each document


In [346]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    tokens_out = []
    for lst in texts:
        doc = nlp(" ".join(lst)) 
        lemma_tokens=[]
        for token in doc:
            if token.text == 'not':
                lemma_tokens.append(token.text)
            else:
                if token.pos_ in allowed_postags:
                    lemma_tokens.append(token.lemma_)
        tokens_out.append(lemma_tokens)
    return tokens_out

In [347]:
trial = [['ordered','carbon','not','scared'], ['hello', 'kitty', 'kitties', 'not']]
lemmatization(trial)

[['order', 'carbon', 'not', 'scare'], ['kitty', 'kitty', 'not']]

In [356]:
#lemmatizing
tokens_lemmatized= lemmatization(c_tokens)

In [315]:
type(tokens_lemmatized)

list

In [357]:
tokens_lemmatized[7]

['order',
 'carbon',
 'store',
 'pick',
 'ton',
 'stuff',
 'always',
 'try',
 'use',
 'top',
 'pair',
 'skirt',
 'pant',
 'go',
 'color',
 'really',
 'nice',
 'charcoal',
 'shimmer',
 'go',
 'well',
 'pencil',
 'skirt',
 'flare',
 'pant',
 'compaint',
 'bit',
 'big',
 'sleeve',
 'long',
 'not',
 'go',
 'petite',
 'also',
 'bit',
 'loose',
 'xxs',
 'keep',
 'wil',
 'ldecide',
 'later',
 'light',
 'color',
 'already',
 'sell',
 'small',
 'size']

In [365]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(tokens_lemmatized, min_count=1) # higher threshold fewer phrases.

#trigram = gensim.models.Phrases(bigram[data_words], threshold=3)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
#trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

In [307]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [366]:
data_words_bigrams = make_bigrams(tokens_lemmatized)

In [364]:
corpus[7]

"I ordered this in carbon for store pick up, and had a ton of stuff (as always) to try on and used this top to pair (skirts and pants). everything went with it. the color is really nice charcoal with shimmer, and went well with pencil skirts, flare pants, etc. my only compaint is it is a bit big, sleeves are long and it doesn't go in petite. also a bit loose for me, but no xxs... so i kept it and wil ldecide later since the light color is already sold out in hte smallest size..."

In [368]:
import nltk
from nltk.util import ngrams
 
# Function to generate n-grams from sentences.
def extract_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]
 
data = 'A class is a blueprint for the object.'
 
print("1-gram: ", extract_ngrams(data, 1))
print("2-gram: ", extract_ngrams(data, 2))
print("3-gram: ", extract_ngrams(data, 3))
print("4-gram: ", extract_ngrams(data, 4))

1-gram:  ['A', 'class', 'is', 'a', 'blueprint', 'for', 'the', 'object', '.']
2-gram:  ['A class', 'class is', 'is a', 'a blueprint', 'blueprint for', 'for the', 'the object', 'object .']
3-gram:  ['A class is', 'class is a', 'is a blueprint', 'a blueprint for', 'blueprint for the', 'for the object', 'the object .']
4-gram:  ['A class is a', 'class is a blueprint', 'is a blueprint for', 'a blueprint for the', 'blueprint for the object', 'for the object .']


In [369]:
from textblob import TextBlob
 
# Function to generate n-grams from sentences.
def extract_ngrams(data, num):
    n_grams = TextBlob(data).ngrams(num)
    return [ ' '.join(grams) for grams in n_grams]
 
data = 'A class is a blueprint for the object.'
 
print("1-gram: ", extract_ngrams(data, 1))
print("2-gram: ", extract_ngrams(data, 2))
print("3-gram: ", extract_ngrams(data, 3))
print("4-gram: ", extract_ngrams(data, 4))

1-gram:  ['A', 'class', 'is', 'a', 'blueprint', 'for', 'the', 'object']
2-gram:  ['A class', 'class is', 'is a', 'a blueprint', 'blueprint for', 'for the', 'the object']
3-gram:  ['A class is', 'class is a', 'is a blueprint', 'a blueprint for', 'blueprint for the', 'for the object']
4-gram:  ['A class is a', 'class is a blueprint', 'is a blueprint for', 'a blueprint for the', 'blueprint for the object']


In [370]:
documents = [row + list(map(lambda ng: '-'.join(ng), ngrams(row, 2))) for row in tokens_lemmatized]

In [ ]:
documents